In [4]:
import os
from dotenv import load_dotenv

FLASK_PORT = 8080
CLOUD_URL = 'https://pulse-fi.herokuapp.com'

load_dotenv()

def getSecret(secret):
    return os.environ.get(secret)

def isRunningInCloud():
    return os.environ.get('RUNNING') == 'cloud'

def baseUrl():
    if isRunningInCloud():
        return CLOUD_URL
    else:
        return 'http://127.0.0.1:'+str(FLASK_PORT)
    
import requests

# url = baseUrl() + '/heartRate'
# response = requests.get(url)

In [97]:
df = get_dataframe_from_query("SELECT heart_rate.* FROM heart_rate;")
df
# get_dataframe_from_query("SELECT music_history.* FROM music_history;")

,user_id,datetime,bpm
0,1,2023-04-12 00:00:00,51
1,1,2023-04-12 00:00:15,51
2,1,2023-04-12 00:00:20,52
3,1,2023-04-12 00:00:35,52
4,1,2023-04-12 00:00:40,53
...,...,...,...
20933,1,2023-04-13 20:03:29,70
20934,1,2023-04-13 20:03:44,69
20935,1,2023-04-13 20:03:54,68
20936,1,2023-04-13 20:04:09,68


In [96]:
import sys 
import os
sys.path.append('../..')
from src import getSecret
from src.sql import get_dataframe_from_query, db_connection

df = get_dataframe_from_query("SELECT music_history.*, songs.* FROM music_history JOIN songs ON music_history.song_id = songs.song_id WHERE user_id = {}".format(1))
temp =df['song_id'].iloc[:,0]
df = df.drop(columns=['song_id'])
df.insert(0, 'song_id', temp)
# display(df)
# df['song_id'].unique()

with db_connection() as conn:
    cur = conn.cursor()
    cur.execute("delete from heart_rate where user_id = 1")
    conn.commit()


In [73]:
import psycopg2
conn = psycopg2.connect(
    host=getSecret('SQL_Host'),
    port=getSecret('SQL_Port'),
    user=getSecret('SQL_User'),
    password=getSecret('SQL_Password'),
    database=getSecret('SQL_Database')
)
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_name = 'users';")
rows = cur.fetchall()
for row in rows:
    print(row)

cur.execute("SELECT COUNT(*) FROM users")
# cur.execute("Delete from users")
count = cur.fetchone()[0]
print("Number of records in users table:", count)

# # list all tables
# cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
# rows = cur.fetchall()
# for row in rows:
#     print(row)

# # fix the query
# cur.execute("SELECT * FROM users")
# rows = cur.fetchall()
# for row in rows:
#     print(row)

# # cur.close()
# # conn.close()

('users',)


ProgrammingError: no results to fetch

In [58]:
import sys
import os
sys.path.append("..")
import pandas as pd

df_heartRate = pd.read_pickle('data/df_heartRate_2023_04_02_to_2023_04_11.pkl')
df_heartRate['datetime'] = pd.to_datetime(df_heartRate['date']) + pd.to_timedelta(df_heartRate['time'])
df_heartRate.rename(columns={'value':'bpm'}, inplace=True)

In [78]:
df_heartRate

import psycopg2

# connect to the database
conn = psycopg2.connect(
    host=getSecret('SQL_Host'),
    port=getSecret('SQL_Port'),
    user=getSecret('SQL_User'),
    password=getSecret('SQL_Password'),
    database=getSecret('SQL_Database')
)

# create a cursor object
cur = conn.cursor()



# execute a query to select all rows from the "Users" table
# cur.execute("SELECT * FROM public.Users")
# cur.execute("SELECT table_schema, table_name FROM information_schema.tables WHERE table_name = 'Users';")
# cur.execute("SET search_path TO public; SELECT * FROM Users")
# cur.execute('SELECT * FROM public.users;')

# for index, row in df_heartRate.iterrows():
#     cur.execute("INSERT INTO public.heart_rate (user_id, bpm, datetime) VALUES (%s, %s, %s)", (1, row['bpm'], row['datetime']))
#     conn.commit()

# cur.execute('SELECT * FROM public.heart_rate;')
# # fetch all rows and print them
# import pandas as pd
# df = pd.DataFrame(cur.fetchall())
# df.columns = [desc[0] for desc in cur.description]
# display(df)
# cur.execute("SELECT music_history.*, songs.* FROM music_history JOIN songs ON music_history.song_id = songs.song_id WHERE user_id = {};".format(1))
cur.execute("SELECT * FROM public.heart_rate;")
# commit
display(pd.DataFrame(cur.fetchall()))
# for row in cur.fetchall():
#     print(row)
# conn.commit()

# df_heartRate = pd.read_sql_query('select * from heartrate where user_id = ' + str(1), con=self.db.engine)
# cur.execute('SELECT * FROM public.music_history;')
# delete 

# cur.execute('SELECT * FROM public.heart_rate;')
# fetch all rows and print them
# rows = cur.fetchall()
# for row in rows:
#     print(row)

# close the cursor and database connection
cur.close()
conn.close()

""


In [27]:
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_name = 'users';")
rows = cur.fetchall()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [21]:
import os
from src import getSecret, isRunningInCloud
# Grabs the folder where the script runs.

DEBUG = True
# Secret key for session management. You can generate random strings here: https://randomkeygen.com/
SECRET_KEY = "o'lEd~n48G[3&@XVF2*]u1`VPF7P%I%,:OA@wuI`.5|%$4neB>h{q=S1N<R5AKL"

if isRunningInCloud():
    SQLALCHEMY_DATABASE_URI = os.environ['DATABASE_URL']
else:
    SQL_username = str(getSecret('SQL_User'))
    SQL_password = str(getSecret('SQL_Password'))
    SQL_host = str(getSecret('SQL_Host'))
    SQL_port = str(getSecret('SQL_Port'))
    SQL_database = str(getSecret('SQL_Database'))
    # Connect to the database
    SQLALCHEMY_DATABASE_URI = 'postgresql://' + SQL_username + ':' + SQL_password + '@' + SQL_host + ':' + SQL_port + '/' + SQL_database
    # 'sqlite:///' + os.path.join(basedir, 'database.db')

    # # Secret key for session management. You can generate random strings here:
    # # https://randomkeygen.com/
    # SECRET_KEY = "o'lEd~n48G[3&@XVF2*]u1`VPF7P%I%,:OA@wuI`.5|%$4neB>h{q=S1N<R5AKL"
    # # Connect to the database
    # SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'database.db')
